In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

# CSVファイルを読み込みます。ファイル名は適宜変更してください。
csv_file = 'train.csv'
data = pd.read_csv(csv_file)

# カテゴリカルデータをマッピングします
data['BusinessTravel'] = data['BusinessTravel'].map({'Non-Travel': 1, 'Travel_Frequently': 2, 'Travel_Rarely': 3})
data['Department'] = data['Department'].map({'Human Resources': 1, 'Research & Development': 2, 'Sales': 3})
data['EducationField'] = data['EducationField'].map({'HR': 1, 'Life Sciences': 2, 'Marketing': 3, 'Medical Sciences': 4, 'Other': 5, 'Technical Degree': 6})
data['Gender'] = data['Gender'].map({'Female': 1, 'Male': 2})
data['MaritalStatus'] = data['MaritalStatus'].map({'Divorced': 1, 'Married': 2, 'Single': 3})
data['Over18'] = data['Over18'].map({'Y': 1, 'N': 2})
data['OverTime'] = data['OverTime'].map({'No': 1, 'Yes': 2})

# 14列目のカテゴリカルデータを数値にマッピングしてカウントします
from collections import Counter

def map_and_count_objects(data_column):
    counter = Counter(data_column)
    mapping = {obj: idx+1 for idx, (obj, count) in enumerate(counter.items())}
    return data_column.map(mapping)

data['JobRole'] = map_and_count_objects(data['JobRole'])

# データを特徴量と目的変数に分割します。
X = data.iloc[:, 1:32]  # 2列目から32列目までが特徴量
y = data.iloc[:, 32]    # 33列目が目的変数

# パイプラインを作成します
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', RandomForestClassifier(random_state=42))
])

# データをトレーニング用とテスト用に分割します。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Random Forestの分類器を作成します。
random_forest = RandomForestClassifier(random_state=40)

# ハイパーパラメータのグリッドを指定します。
param_grid = {
    'imputer__strategy': ['mean', 'median', 'most_frequent', 'constant'],
    'classifier__n_estimators': [50, 75, 100, 150, 200],
    'classifier__max_depth': [None, 10, 20, 30]
}

# GridSearchCVを作成します。
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# グリッドサーチを実行してモデルをトレーニングします。
grid_search.fit(X_train, y_train)

# 最適なハイパーパラメータを表示します。
print("Best Parameters:", grid_search.best_params_)

# 最適なモデルを取得します。
best_model = grid_search.best_estimator_

# テストデータで予測を行います。
y_pred = best_model.predict(X_test)

# 正解率を評価します。
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# その他の評価指標を出力します（適合率、再現率、F1スコアなど）。
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Best Parameters: {'classifier__max_depth': None, 'classifier__n_estimators': 100, 'imputer__strategy': 'median'}
Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.97      0.90       193
           1       0.71      0.26      0.37        47

    accuracy                           0.83       240
   macro avg       0.77      0.61      0.64       240
weighted avg       0.82      0.83      0.80       240



In [16]:
# テストデータ用のCSVファイルを読み込みます。
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file)

# テストデータのカテゴリカルデータをマッピングします
test_data['BusinessTravel'] = test_data['BusinessTravel'].map({'Non-Travel': 1, 'Travel_Frequently': 2, 'Travel_Rarely': 3})
test_data['Department'] = test_data['Department'].map({'Human Resources': 1, 'Research & Development': 2, 'Sales': 3})
test_data['EducationField'] = test_data['EducationField'].map({'HR': 1, 'Life Sciences': 2, 'Marketing': 3, 'Medical Sciences': 4, 'Other': 5, 'Technical Degree': 6})
test_data['Gender'] = test_data['Gender'].map({'Female': 1, 'Male': 2})
test_data['MaritalStatus'] = test_data['MaritalStatus'].map({'Divorced': 1, 'Married': 2, 'Single': 3})
test_data['Over18'] = test_data['Over18'].map({'Y': 1, 'N': 2})
test_data['OverTime'] = test_data['OverTime'].map({'No': 1, 'Yes': 2})

# 14列目のカテゴリカルデータを数値にマッピングしてカウントします
test_data['JobRole'] = map_and_count_objects(test_data['JobRole'])

# テストデータで予測を行います。
y_pred = best_model.predict(test_data.iloc[:, 1:])  # 2列目から32列目までが特徴量

# 推定結果をCSVファイルとして保存します。
output_csv_file = 'predicted_classes2.csv'
result_df = pd.DataFrame({'id': test_data.iloc[:, 0], 'class': y_pred})
result_df.to_csv(output_csv_file, index=False, header=False)
